In [1]:
import pyforest 

In [48]:
!pip install gensim

     |████████████████████████████████| 24.0 MB 118 kB/s ta 0:00:013    |████████▏                       | 6.1 MB 194 kB/s eta 0:01:32     |████████████████████████▌       | 18.4 MB 23 kB/s eta 0:03:57     |████████████████████████████▍   | 21.4 MB 82 kB/s eta 0:00:33     |█████████████████████████████   | 21.8 MB 59 kB/s eta 0:00:38
     |████████████████████████████████| 58 kB 57 kB/s eta 0:00:01


In [8]:
import matplotlib.pyplot as plt
import re
import os
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
#from sklearn import confusion_matrix
import gensim

In [10]:
df = pd.read_csv('/Users/brianrochford/Downloads/MovieReview-Sample.csv', header = None, names = ['Text', 'label'])

<IPython.core.display.Javascript object>

In [117]:
df = pd.read_table("/Users/brianrochford/Downloads/MovieReview-Sample.csv",header = None, names = ['Text', 'label'])


In [118]:
df

,Text,label
0,"1,""films adapted from comic books have had ple...",NaN
1,"2,""every now and then a movie comes along from...",NaN
2,"3,""you've got mail works alot better than it d...",NaN
3,"4,"" jaws is a rare film that grabs your att...",NaN
4,"5,""moviemaking is a lot like being the general...",NaN
...,...,...
1995,"1996,""if anything , stigmata should be tak...",NaN
1996,"1997,""john boorman's zardoz is a goofy cin...",NaN
1997,"1998,""the kids in the hall are an acquired tas...",NaN
1998,"1999,""there was a time when john carpenter was...",NaN


In [98]:
import nltk
nltk.download("opinion_lexicon")

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/brianrochford/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

## Word processing

In [119]:
# remove non alphabets
remove_non_alphabets = lambda x: re.sub(r'[^a-zA-Z]',' ',x)

# tokenn alphabets-only list
tokenize = lambda x: word_tokenize(x)

# assign ps to a lambda function to run on each line of value
ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]

# assign lemmatizer to a lambda function to run on each line of value
lemmatizer = WordNetLemmatizer()
leammtizer = lambda x: [ lemmatizer.lemmatize(word) for word in x ]

In [120]:
# apply all above methods to the column ''
print('Processing : [=', end='')
df['Text'] = df['Text'].apply(remove_non_alphabets)
print('=', end='')
df['Text'] = df['Text'].apply(tokenize)
print('=', end='')
df['Text'] = df['Text'].apply(stem)
print('=', end='')
df['Text'] = df['Text'].apply(leammtizer)
print('=', end='')
df['Text'] = df['Text'].apply(lambda x: ' '.join(x))
print('] : Completed', end='')
df.head()

Processing : [=====] : Completed

,Text,label
0,film adapt from comic book have had plenti of ...,NaN
1,everi now and then a movi come along from a su...,NaN
2,you ve got mail work alot better than it deser...,NaN
3,jaw is a rare film that grab your attent befor...,NaN
4,moviemak is a lot like be the gener manag of a...,NaN


In [6]:
from nltk.corpus import opinion_lexicon


In [108]:
pos_list_BL = set(opinion_lexicon.positive())
neg_list_BL=set(opinion_lexicon.negative())

In [128]:
df2 = df

In [129]:
data = df.Text.str.lower()
data2 = df2.Text.str.lower()

In [110]:
df['Text'] = df.Text.str.lower()

In [111]:
df['Text']


1       film adapt from comic book have had plenti of ...
2       everi now and then a movi come along from a su...
3       you ve got mail work alot better than it deser...
4       jaw is a rare film that grab your attent befor...
5       moviemak is a lot like be the gener manag of a...
                              ...                        
1996    if anyth stigma should be taken a a warn again...
1997    john boorman s zardoz is a goofi cinemat debac...
1998    the kid in the hall are an acquir tast it took...
1999    there wa a time when john carpent wa a great h...
2000    two parti guy bob their head to haddaway s dan...
Name: Text, Length: 2000, dtype: object

In [107]:
data

1       film adapt from comic book have had plenti of ...
2       everi now and then a movi come along from a su...
3       you ve got mail work alot better than it deser...
4       jaw is a rare film that grab your attent befor...
5       moviemak is a lot like be the gener manag of a...
                              ...                        
1996    if anyth stigma should be taken a a warn again...
1997    john boorman s zardoz is a goofi cinemat debac...
1998    the kid in the hall are an acquir tast it took...
1999    there wa a time when john carpent wa a great h...
2000    two parti guy bob their head to haddaway s dan...
Name: Text, Length: 2000, dtype: object

## Dictionary Function

In [122]:
def count_pos_neg(data, positive_dict, negative_dict):
# count of positive and negative words that appeared in each message
# net count which is calculated by positive count subtracting negative count. 
    poscnt = []
    negcnt = []
    netcnt = []

    for nrow in range(0,len(data)):
        text = data[nrow]
        
        qa = 0
        qb = 0

        for word in positive_dict :
            if (word in text) :
                qa = qa + 1

        for word in negative_dict :
            if (word in text) :
                qb = qb + 1

        qc = qa - qb

        poscnt.append(qa)
        negcnt.append(qb)
        netcnt.append(qc)

    return (poscnt, negcnt, netcnt)

## Bing Liu Dictionary

In [115]:
df.iloc[:,1]

1       1
2       1
3       1
4       1
5       1
       ..
1996    0
1997    0
1998    0
1999    0
2000    0
Name: label, Length: 2000, dtype: int64

In [123]:
df['poscnt_BL'], df['negcnt_BL'], df['netcnt_BL'] = count_pos_neg(data, pos_list_BL, neg_list_BL)

In [124]:
df

,Text,label,poscnt_BL,negcnt_BL,netcnt_BL
0,film adapt from comic book have had plenti of ...,NaN,12,24,-12
1,everi now and then a movi come along from a su...,NaN,10,25,-15
2,you ve got mail work alot better than it deser...,NaN,19,14,5
3,jaw is a rare film that grab your attent befor...,NaN,10,22,-12
4,moviemak is a lot like be the gener manag of a...,NaN,13,19,-6
...,...,...,...,...,...
1995,if anyth stigma should be taken a a warn again...,NaN,7,28,-21
1996,john boorman s zardoz is a goofi cinemat debac...,NaN,13,15,-2
1997,the kid in the hall are an acquir tast it took...,NaN,12,14,-2
1998,there wa a time when john carpent wa a great h...,NaN,14,25,-11


In [25]:
df.to_csv('textAnalyticsHw2.csv')

## LM Dictionary

In [125]:
def read_local_dictionary(file):
    # create dictionary list
    words_dict = []
    with open(file, "r") as f: 
        for line in f:
            t = line.strip().lower()
            words_dict.append(t)
    return words_dict

In [126]:
pos_list_LM = read_local_dictionary('/Users/brianrochford/Downloads/positive-words-LM.txt')
neg_list_LM = read_local_dictionary('/Users/brianrochford/Downloads/negative-words-LM.txt')

In [130]:
df2['poscnt_LM'], df2['negcnt_LM'], df2['netcnt_LM'] = count_pos_neg(data2, pos_list_LM, neg_list_LM)

In [131]:
df2[['Text','poscnt_LM','negcnt_LM','netcnt_LM']].head(5)

,Text,poscnt_LM,negcnt_LM,netcnt_LM
0,film adapt from comic book have had plenti of ...,3,6,-3
1,everi now and then a movi come along from a su...,3,6,-3
2,you ve got mail work alot better than it deser...,8,5,3
3,jaw is a rare film that grab your attent befor...,2,8,-6
4,moviemak is a lot like be the gener manag of a...,0,4,-4


## TextBlob

In [44]:
from textblob import TextBlob

In [132]:
df2["score_TextBlob"] = df2["Text"].map(lambda x:TextBlob(x).sentiment.polarity)




In [133]:
df2[["Text","score_TextBlob"]].head(5)

,Text,score_TextBlob
0,film adapt from comic book have had plenti of ...,0.029550
1,everi now and then a movi come along from a su...,0.240473
2,you ve got mail work alot better than it deser...,0.122305
3,jaw is a rare film that grab your attent befor...,0.176044
4,moviemak is a lot like be the gener manag of a...,0.134746


## Vader

In [78]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [134]:
analyzer = SentimentIntensityAnalyzer()
scores = [analyzer.polarity_scores(sentence) for sentence in data]

In [135]:
neg_s = [i["neg"] for i in scores]
neu_s = [i["neu"] for i in scores]
pos_s = [i["pos"] for i in scores]
compound_s = [i["compound"] for i in scores]

In [136]:
df['negscore_Vader'], df['neuscore_Vader'], df['posscore_Vader'], df['compound_Vader'] = neg_s, neu_s, pos_s, compound_s

In [137]:
df[['Text','negscore_Vader','neuscore_Vader','posscore_Vader','compound_Vader']].head(5)


,Text,negscore_Vader,neuscore_Vader,posscore_Vader,compound_Vader
0,film adapt from comic book have had plenti of ...,0.121,0.803,0.076,-0.9754
1,everi now and then a movi come along from a su...,0.061,0.828,0.111,0.9524
2,you ve got mail work alot better than it deser...,0.050,0.802,0.147,0.9890
3,jaw is a rare film that grab your attent befor...,0.068,0.849,0.084,0.7269
4,moviemak is a lot like be the gener manag of a...,0.043,0.846,0.112,0.9765


In [138]:
df.to_excel("textAnalyticsHw2new.xlsx")

In [2]:
df_scores = pd.read_excel('textAnalyticsHw2new.xlsx')

<IPython.core.display.Javascript object>

In [30]:
df_scores

,Unnamed: 0,Text,label,poscnt_BL,negcnt_BL,netcnt_BL,poscnt_LM,negcnt_LM,netcnt_LM,score_TextBlob,negscore_Vader,neuscore_Vader,posscore_Vader,compound_Vader,SentLabel,blobBinary,vaderBinary,blBinary,lmBinary
0,0,film adapt from comic book have had plenti of ...,NaN,12,24,-12,3,6,-3,0.029550,0.121,0.803,0.076,-0.9754,1,1,0,0,0
1,1,everi now and then a movi come along from a su...,NaN,10,25,-15,3,6,-3,0.240473,0.061,0.828,0.111,0.9524,1,1,1,0,0
2,2,you ve got mail work alot better than it deser...,NaN,19,14,5,8,5,3,0.122305,0.050,0.802,0.147,0.9890,1,1,1,1,1
3,3,jaw is a rare film that grab your attent befor...,NaN,10,22,-12,2,8,-6,0.176044,0.068,0.849,0.084,0.7269,1,1,1,0,0
4,4,moviemak is a lot like be the gener manag of a...,NaN,13,19,-6,0,4,-4,0.134746,0.043,0.846,0.112,0.9765,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,if anyth stigma should be taken a a warn again...,NaN,7,28,-21,2,8,-6,-0.064632,0.112,0.835,0.053,-0.9661,0,0,0,0,0
1996,1996,john boorman s zardoz is a goofi cinemat debac...,NaN,13,15,-2,1,3,-2,0.107353,0.026,0.908,0.066,0.8937,0,1,1,0,0
1997,1997,the kid in the hall are an acquir tast it took...,NaN,12,14,-2,2,8,-6,0.202189,0.045,0.867,0.087,0.9610,0,1,1,0,0
1998,1998,there wa a time when john carpent wa a great h...,NaN,14,25,-11,5,10,-5,0.109127,0.117,0.764,0.119,-0.4841,0,1,0,0,0


## TextBlob precision calculation

In [3]:
df_scores['blobBinary'] = ''
df_scores.loc[df_scores.score_TextBlob > 0, 'blobBinary'] = 1
df_scores.loc[df_scores.score_TextBlob <= 0, 'blobBinary'] = 0

knownResult = df_scores.SentLabel.tolist()
textBlobResult = df_scores.blobBinary.tolist()

In [4]:
knownResult = df_scores.SentLabel.tolist()
textBlobResult = df_scores.blobBinary.tolist()

In [10]:
import pyforest
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [11]:
print(confusion_matrix(knownResult, textBlobResult))

[[253 748]
 [112 887]]


In [14]:
textBlobReport = (classification_report(knownResult, textBlobResult))
print(textBlobReport)

              precision    recall  f1-score   support

           0       0.69      0.25      0.37      1001
           1       0.54      0.89      0.67       999

    accuracy                           0.57      2000
   macro avg       0.62      0.57      0.52      2000
weighted avg       0.62      0.57      0.52      2000



## Vader Report

In [18]:
df_scores['vaderBinary'] = ''
df_scores.loc[df_scores.compound_Vader > 0, 'vaderBinary'] = 1
df_scores.loc[df_scores.compound_Vader <= 0, 'vaderBinary'] = 0

knownResult_v = df_scores.SentLabel.tolist()
vaderResult = df_scores.vaderBinary.tolist()

In [19]:
print(confusion_matrix(knownResult_v, vaderResult))

[[421 580]
 [250 749]]


In [20]:
vaderReport = (classification_report(knownResult_v, vaderResult))
print(vaderReport)

              precision    recall  f1-score   support

           0       0.63      0.42      0.50      1001
           1       0.56      0.75      0.64       999

    accuracy                           0.58      2000
   macro avg       0.60      0.59      0.57      2000
weighted avg       0.60      0.58      0.57      2000



## Bing Liu Dictionary Report

In [21]:
df_scores['blBinary'] = ''
df_scores.loc[df_scores.netcnt_BL > 0, 'blBinary'] = 1
df_scores.loc[df_scores.netcnt_BL <= 0, 'blBinary'] = 0

knownResult_bl = df_scores.SentLabel.tolist()
bingLiuResult = df_scores.blBinary.tolist()

In [22]:
print(confusion_matrix(knownResult_bl, bingLiuResult))

[[962  39]
 [894 105]]


In [23]:
bingLiuReport = (classification_report(knownResult_bl, bingLiuResult))
print(bingLiuReport)

              precision    recall  f1-score   support

           0       0.52      0.96      0.67      1001
           1       0.73      0.11      0.18       999

    accuracy                           0.53      2000
   macro avg       0.62      0.53      0.43      2000
weighted avg       0.62      0.53      0.43      2000



## LM Dictionary Report

In [24]:
df_scores['lmBinary'] = ''
df_scores.loc[df_scores.netcnt_LM > 0, 'lmBinary'] = 1
df_scores.loc[df_scores.netcnt_LM <= 0, 'lmBinary'] = 0

knownResult_lm = df_scores.SentLabel.tolist()
lmResult = df_scores.lmBinary.tolist()

In [25]:
print(confusion_matrix(knownResult_lm, lmResult))

[[947  54]
 [861 138]]


In [26]:
lmReport = (classification_report(knownResult_lm, lmResult))
print(lmReport)

              precision    recall  f1-score   support

           0       0.52      0.95      0.67      1001
           1       0.72      0.14      0.23       999

    accuracy                           0.54      2000
   macro avg       0.62      0.54      0.45      2000
weighted avg       0.62      0.54      0.45      2000



## Question 2 Ensemble

In [29]:
from sklearn.naive_bayes import MultinomialNB # import naive bayes
from sklearn.tree import DecisionTreeClassifier # import Decision Tree
from sklearn.ensemble import RandomForestClassifier # import random forest

## Ensemble Model

In [35]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV

In [89]:
i = 0
df_scores['ensemble_score'] = ''
while i < 2000:
    sum_scores = df_scores['blobBinary'][i] + df_scores['vaderBinary'][i] + df_scores['blBinary'][i]
    if sum_scores >= 2:
        df_scores['ensemble_score'][i] = 1
    elif sum_scores < 2:
        df_scores['ensemble_score'][i] = 0
    i = i +1
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [95]:
#df_scores

In [91]:
knownResult_e = df_scores.SentLabel.tolist()
ensembleResult = df_scores.ensemble_score.tolist()

In [92]:
print(confusion_matrix(knownResult_e, ensembleResult))

[[490 511]
 [287 712]]


In [94]:
ensembleReport = (classification_report(knownResult_e, ensembleResult))
print(ensembleReport)

              precision    recall  f1-score   support

           0       0.63      0.49      0.55      1001
           1       0.58      0.71      0.64       999

    accuracy                           0.60      2000
   macro avg       0.61      0.60      0.60      2000
weighted avg       0.61      0.60      0.60      2000

